# Task 4 AML : Sleeping Staging

## Imports

In [2]:
import sys
!{sys.executable} -m pip install mne

The directory '/home/atroska/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/atroska/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 19.0.3, however version 20.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import mne
from tqdm import tqdm
from scipy import signal
from mne.time_frequency import psd_welch
from mne.time_frequency import tfr_stockwell
from mne import EpochsArray,create_info
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import balanced_accuracy_score

DEBUG:matplotlib:$HOME=/home/atroska
DEBUG:matplotlib:CONFIGDIR=/home/atroska/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'op

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:matplotlib:CACHEDIR=/home/atroska/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/atroska/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Read Data

In [4]:
train_eeg1 = pd.read_csv("train_eeg1.csv")
train_eeg2 = pd.read_csv("train_eeg2.csv")
train_emg = pd.read_csv("train_emg.csv")
train_labels = pd.read_csv("train_labels.csv")

test_eeg1 = pd.read_csv("test_eeg1.csv")
test_eeg2 = pd.read_csv("test_eeg2.csv")
test_emg = pd.read_csv("test_emg.csv")

sample = pd.read_csv("sample.csv")

In [5]:
def from_csv_to_ndarray(data):
    """
    Fransfer data from pd.DataFrame to ndarray for later model training
    :param data: data in pd.DataFrame
    :return ndarray: data in ndarray
    """
    data.head()
    ndarray = data.values
    if ndarray.shape[1] == 2:
        return ndarray[:, 1]
    else:
        return ndarray[:, 1:]

In [6]:
#train_eeg1_numpy=np.expand_dims(train_eeg1.to_numpy()[:,1:],-1)
#train_eeg2_numpy=np.expand_dims(train_eeg2.to_numpy()[:,1:],-1)
#train_emg_numpy=np.expand_dims(train_emg.to_numpy()[:,1:],-1)
#train_labels_numpy=train_labels.to_numpy()[:,1:]

#test_eeg1_numpy=np.expand_dims(test_eeg1.to_numpy()[:,1:],-1)
#test_eeg2_numpy=np.expand_dims(test_eeg2.to_numpy()[:,1:],-1)
#test_emg_numpy=np.expand_dims(test_emg.to_numpy()[:,1:],-1)

train_eeg1_numpy=from_csv_to_ndarray(train_eeg1)
train_eeg2_numpy=from_csv_to_ndarray(train_eeg2)
train_emg_numpy=from_csv_to_ndarray(train_emg)
train_labels_numpy=from_csv_to_ndarray(train_labels)

test_eeg1_numpy=from_csv_to_ndarray(test_eeg1)
test_eeg2_numpy=from_csv_to_ndarray(test_eeg1)
test_emg_numpy=from_csv_to_ndarray(test_eeg1)


# Pre-process data

In [6]:
def extract_and_save_fft(signals, name, mode, cut_len):
    print('Processing signal {} {}...'.format(mode, name))
    if mode == 'train':
        sub_num = 3
    else:
        sub_num = 2
    fft_features = []
    for i in range(sub_num):
        sub_signals = signals[i*21600:(i+1)*21600]
        sig_pad_1 = sub_signals[0, :240].copy()
        sig_pad_2 = sub_signals[-1, -240:].copy()
        sig_concat = sub_signals.flatten()
        sig_concat = np.concatenate((sig_pad_1, sig_concat, sig_pad_2))
        f, t, Sxx = signal.spectrogram(sig_concat, fs=128, window='hamming', noverlap=256 - 16, nperseg=256)
        Sxx = Sxx[1:cut_len + 1, :]
        for i in range(21600):
            fft_features.append(Sxx[:, i*32:(i+1)*32])
    fft_features = np.array(fft_features)
    np.save('{}_{}_fft_features_new.npy'.format(mode, name), fft_features)
    return fft_features

In [7]:
fft_train_eeg1 = extract_and_save_fft(train_eeg1_numpy, 'eeg1', 'train', 48)
fft_train_eeg2 = extract_and_save_fft(train_eeg2_numpy, 'eeg2', 'train', 48)
fft_train_emg = extract_and_save_fft(train_emg_numpy, 'emg', 'train', 60)

fft_test_eeg1 = extract_and_save_fft(test_eeg1_numpy, 'eeg1', 'test', 48)
fft_test_eeg2 = extract_and_save_fft(test_eeg2_numpy, 'eeg2', 'test', 48)
fft_test_emg = extract_and_save_fft(test_emg_numpy, 'emg', 'test', 60)

Processing signal train eeg1...
Processing signal train eeg2...
Processing signal train emg...
Processing signal test eeg1...
Processing signal test eeg2...
Processing signal test emg...


In [8]:
fft_train_eeg1 = np.log(fft_train_eeg1)
fft_train_eeg2 = np.log(fft_train_eeg2)
fft_train_emg = np.log(fft_train_emg)

fft_test_eeg1 = np.log(fft_test_eeg1)
fft_test_eeg2 = np.log(fft_test_eeg2)
fft_test_emg = np.log(fft_test_emg)

In [9]:
features_train = []
for eeg1_sig, eeg2_sig, emg_sig in tqdm(zip(fft_train_eeg1, fft_train_eeg2, fft_train_emg)):
    eeg1_sig = (eeg1_sig - np.mean(eeg1_sig, axis=0, keepdims=True)) / np.std(eeg1_sig, axis=0, keepdims=True)
    eeg2_sig = (eeg2_sig - np.mean(eeg2_sig, axis=0, keepdims=True)) / np.std(eeg2_sig, axis=0, keepdims=True)
    emg_sig = np.sum(emg_sig, axis=0, keepdims=True)
    emg_sig = emg_sig.repeat(eeg1_sig.shape[0], axis=0)
    eeg1_sig = eeg1_sig.reshape((eeg1_sig.shape[0], eeg1_sig.shape[1], 1))
    eeg2_sig = eeg2_sig.reshape((eeg1_sig.shape[0], eeg1_sig.shape[1], 1))
    emg_sig = emg_sig.reshape((eeg1_sig.shape[0], eeg1_sig.shape[1], 1))
    features_train.append(np.concatenate((eeg1_sig, eeg2_sig, emg_sig), axis=2))
features_train = np.array(features_train)
np.save('fft_features_train.npy', features_train)



64800it [00:07, 8907.55it/s]


In [10]:
features_test = []
for eeg1_sig, eeg2_sig, emg_sig in tqdm(zip(fft_train_eeg1, fft_train_eeg2, fft_train_emg)):
    eeg1_sig = (eeg1_sig - np.mean(eeg1_sig, axis=0, keepdims=True)) / np.std(eeg1_sig, axis=0, keepdims=True)
    eeg2_sig = (eeg2_sig - np.mean(eeg2_sig, axis=0, keepdims=True)) / np.std(eeg2_sig, axis=0, keepdims=True)
    emg_sig = np.sum(emg_sig, axis=0, keepdims=True)
    emg_sig = emg_sig.repeat(eeg1_sig.shape[0], axis=0)
    eeg1_sig = eeg1_sig.reshape((eeg1_sig.shape[0], eeg1_sig.shape[1], 1))
    eeg2_sig = eeg2_sig.reshape((eeg1_sig.shape[0], eeg1_sig.shape[1], 1))
    emg_sig = emg_sig.reshape((eeg1_sig.shape[0], eeg1_sig.shape[1], 1))
    features_test.append(np.concatenate((eeg1_sig, eeg2_sig, emg_sig), axis=2))
features_test = np.array(features_test)
np.save('fft_features_test.npy', features_test)


64800it [00:06, 9847.10it/s] 


In [2]:
import sys
!{sys.executable} -m pip install torchvision

The directory '/home/atroska/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/atroska/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 12.8MB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 776.7MB 140kB/s eta 0:00:01    22% |███████▏                        | 173.5MB 3.8MB/s eta 0:02:41    24% |███████▊                        | 186.6MB 4.1MB/s eta 0:02:25    33% |██████████▊                     | 259.1MB 5.7MB/s eta 0:01:32    37% |████████████                    | 289.0MB 6.0MB/s eta 0:01:22    49% |███████████████▉                | 385.0MB 6.9MB/s eta 0:00:58    53% |█████

In [7]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        """
        x1 = x[:, :2, :, :]
        x2 = x[:, 2, :, :]
        x2 = torch.mean(x2, dim=(1, 2)).view(x.size(0), -1)
        x2[x2>-0.1] = 10
        x2[x2<-0.1] = -10
        """
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        # out = torch.cat((out, x2), dim=1)
        out = self.fc2(out)
        return out


class SleepingDataset(Dataset):
    def __init__(self, x, y):
        # Transforms
        self.to_tensor = transforms.ToTensor()
        self.x = x
        self.y = y-1
        self.len = len(x)

    def __getitem__(self, index):
        img = torch.from_numpy(self.x[index])
        img = img.permute((2, 0, 1))
        return img, self.y[index]

    def __len__(self):
        return self.len

In [11]:
x_all = np.load('fft_features_train.npy')
y_all = train_labels_numpy
np.save('y_all.npy', y_all)

In [13]:
num_epochs = 5
num_classes = 3
batch_size = 128
learning_rate = 0.0007
test_sub = 0

x_all[:21600, :, :, 2] = (x_all[:21600, :, :, 2] - np.mean(x_all[:21600, :, :, 2])) / np.std(x_all[:21600, :, :, 2])
x_all[21600:2*21600, :, :, 2] = (x_all[21600:2*21600, :, :, 2] - np.mean(x_all[21600:2*21600, :, :, 2])) / np.std(x_all[21600:2*21600, :, :, 2])
x_all[:2*21600, :, :, 2] = (x_all[:2*21600, :, :, 2] - np.mean(x_all[:2*21600, :, :, 2] )) / np.std(x_all[:2*21600, :, :, 2] )

if test_sub == 0:
    x_train = x_all[21600:]
    y_train = y_all[21600:]
    x_test = x_all[:21600]
    y_test = y_all[:21600]
elif test_sub == 1:
    x_train = np.concatenate((x_all[21600:], x_all[2*21600:]), axis=0)
    y_train = np.concatenate((y_all[21600:], y_all[2*21600:]), axis=0)
    x_test = x_all[21600:2*21600]
    y_test = y_all[21600:2*21600]
else:
    x_train = x_all[:2*21600]
    y_train = y_all[:2*21600]
    x_test = x_all[2*21600:]
    y_test = y_all[2*21600:]

training_set = SleepingDataset(x_train, y_train)
test_set = SleepingDataset(x_test, y_test)
train_loader = torch.utils.data.DataLoader(dataset=training_set, batch_size=batch_size,  shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

model = Net(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1, 1, 1]).to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [14]:
# Train the model
print('Training..')
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
torch.save(model.state_dict(), './model.ckpt')
# Test the model
print('Evaluating..')
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
y_pred = np.zeros(len(x_test))
y_true = np.zeros(len(x_test))
cnt = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred[cnt:cnt+len(labels)] = predicted.data.cpu().numpy()
        y_true[cnt:cnt+len(labels)] = labels.data.cpu().numpy()
        cnt+=len(labels)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))
print(balanced_accuracy_score(y_true, y_pred))

Training..
Epoch [1/5], Step [100/338], Loss: 0.3027
Epoch [1/5], Step [200/338], Loss: 0.2211
Epoch [1/5], Step [300/338], Loss: 0.2125
Epoch [2/5], Step [100/338], Loss: 0.1847
Epoch [2/5], Step [200/338], Loss: 0.2595
Epoch [2/5], Step [300/338], Loss: 0.2788
Epoch [3/5], Step [100/338], Loss: 0.1626
Epoch [3/5], Step [200/338], Loss: 0.1266
Epoch [3/5], Step [300/338], Loss: 0.2008
Epoch [4/5], Step [100/338], Loss: 0.1381
Epoch [4/5], Step [200/338], Loss: 0.1882
Epoch [4/5], Step [300/338], Loss: 0.2082
Epoch [5/5], Step [100/338], Loss: 0.1460
Epoch [5/5], Step [200/338], Loss: 0.0885
Epoch [5/5], Step [300/338], Loss: 0.1128
Evaluating..
[[10411   799    30]
 [  805  7646   202]
 [   53   189  1465]]
0.8893668474938873


In [15]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        """
        x1 = x[:, :2, :, :]
        x2 = x[:, 2, :, :]
        x2 = torch.mean(x2, dim=(1, 2)).view(x.size(0), -1)
        x2[x2>-0.1] = 10
        x2[x2<-0.1] = -10
        """
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        # out = torch.cat((out, x2), dim=1)
        out = self.fc2(out)
        return out


class SleepingDataset(Dataset):
    def __init__(self, x, y):
        # Transforms
        self.to_tensor = transforms.ToTensor()
        self.x = x
        self.y = y-1
        self.len = len(x)

    def __getitem__(self, index):
        img = torch.from_numpy(self.x[index])
        img = img.permute((2, 0, 1))
        return img, self.y[index]

    def __len__(self):
        return self.len

In [18]:
x_train = np.load('fft_features_train.npy')
y_train = np.load('y_all.npy')
x_test = np.load('fft_features_test.npy')

num_epochs = 5
num_classes = 3
batch_size = 128
learning_rate = 0.0007

training_set = SleepingDataset(x_train, y_train)
test_set = SleepingDataset(x_test, np.zeros(len(x_test)))
train_loader = torch.utils.data.DataLoader(dataset=training_set, batch_size=batch_size,  shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

model = Net(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
print('Training..')
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
torch.save(model.state_dict(), './model.ckpt')
# Test the model
print('Evaluating..')
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
y_pred = np.zeros(len(x_test))
cnt = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred[cnt:cnt+len(labels)] = predicted.data.cpu().numpy()
        cnt+=len(labels)
print(y_pred)
sub = pd.DataFrame()
sub['Id'] = np.arange(len(y_pred))
sub['y'] = y_pred+1
y_pred_save_dir = 'y_test_5.0.csv'
sub.to_csv(y_pred_save_dir, index=False)

Training..
Epoch [1/5], Step [100/507], Loss: 0.3410
Epoch [1/5], Step [200/507], Loss: 0.3237
Epoch [1/5], Step [300/507], Loss: 0.3416
Epoch [1/5], Step [400/507], Loss: 0.1464
Epoch [1/5], Step [500/507], Loss: 0.3460
Epoch [2/5], Step [100/507], Loss: 0.1737
Epoch [2/5], Step [200/507], Loss: 0.3233
Epoch [2/5], Step [300/507], Loss: 0.1774
Epoch [2/5], Step [400/507], Loss: 0.1550
Epoch [2/5], Step [500/507], Loss: 0.2011
Epoch [3/5], Step [100/507], Loss: 0.2120
Epoch [3/5], Step [200/507], Loss: 0.1902
Epoch [3/5], Step [300/507], Loss: 0.0869
Epoch [3/5], Step [400/507], Loss: 0.1062
Epoch [3/5], Step [500/507], Loss: 0.1427
Epoch [4/5], Step [100/507], Loss: 0.2174
Epoch [4/5], Step [200/507], Loss: 0.1931
Epoch [4/5], Step [300/507], Loss: 0.1670
Epoch [4/5], Step [400/507], Loss: 0.2147
Epoch [4/5], Step [500/507], Loss: 0.0868
Epoch [5/5], Step [100/507], Loss: 0.1594
Epoch [5/5], Step [200/507], Loss: 0.1470
Epoch [5/5], Step [300/507], Loss: 0.1696
Epoch [5/5], Step [400/